In [ ]:
import pandas as pd
import numpy as np
from glob import glob 

import cv2
import numpy as np

import pytesseract

"""
POC: Load Analyze-format MRI (.hdr/.img) and HDF5 data in Python
Requires: nibabel, h5py, numpy, matplotlib, pandas
"""

import os
import nibabel as nb
import matplotlib.pyplot as plt
from scipy import ndimage
import torch
from transformers import pipeline
from PIL import Image

2025-05-11 12:23:44.938237: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-11 12:23:45.046597: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746980625.095808   96948 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746980625.107747   96948 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746980625.204509   96948 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

: 

# Loading the data

In [ ]:
# Load data
data_dir = "./data/"
data_l = glob(data_dir + '*.csv')
print(data_l)

event_df = pd.read_csv(data_l[0])
eye_track_df = pd.read_csv(data_l[1])
motion_df = pd.read_csv(data_l[2])

# Monkey K2 (Kuma)
# Electrodes [52 -> 64] and [121 -> 128] (inclusive) are in the medial wall of the macaque brain.
ecog_df = pd.read_csv(data_l[3])


['./data/Event.csv', './data/EyeTrack.csv', './data/Motion.csv', './data/ECoG.csv']


In [ ]:
image_path = "./imgs/K2.png"
raw_image = Image.open(image_path)

# Sulci Notes:
Sulci from right to left
1. Principal Sulcus (25 is at the principal sulcus)
2. Arcuate Sulcus
3. Central Sulcus
4. Intraparietal Sulcus
5. Lateral Sulcus or Sylvian Fissure (21 is at the conjunction of Superior Temporal Sulcus and Lateral Sulcus)
6. Superior Temporal Sulcus
7. Lunate Sulcus
8. Unknown. Possibly Calcarine Sulcus.

In [ ]:
# Example of manually mapping coordinates:
import numpy as np

# Example ECoG electrode positions on macaque brain (F99 space), in mm
macaque_ecog_coords = np.array([
    [10, -15, 5],
    [12, -13, 6],
    [14, -12, 8],
    [16, -10, 9],
])  # You can replace this with your actual data

# Sample activity levels per electrode
activity = np.array([0.2, 0.8, 0.5, 0.9])


In [ ]:
import nibabel as nib
from nilearn import plotting

# Load a macaque surface model (use appropriate path)
lh_surface = nib.load('F99.L.inflated.gii')  # Left hemisphere inflated surface
lh_coords = lh_surface.darrays[0].data
lh_faces = lh_surface.darrays[1].data


In [ ]:
import matplotlib.pyplot as plt
import pyvista as pv
from scipy.spatial import cKDTree

# Build a fast nearest-neighbor mapping from electrode to surface
tree = cKDTree(lh_coords)
_, indices = tree.query(macaque_ecog_coords)

# Create vertex color array
vertex_colors = np.zeros(len(lh_coords))
vertex_colors[indices] = activity

# Visualize using pyvista
plotter = pv.Plotter()
mesh = pv.PolyData(lh_coords, lh_faces)
mesh["Activity"] = vertex_colors
plotter.add_mesh(mesh, scalars="Activity", cmap="hot", show_edges=False)
plotter.add_title("Macaque Brain ECoG Activity")
plotter.show()


In [ ]:
# Fake mapping: just mirror the coordinates to a human surface
human_coords = macaque_ecog_coords * np.array([1.2, 1.3, 1.1]) + np.array([30, 40, 20])


In [ ]:
# Load fsaverage surface (use FreeSurfer or download)
fs_lh = nib.load('fsaverage/surf/lh.inflated.gii')
fs_coords = fs_lh.darrays[0].data
fs_faces = fs_lh.darrays[1].data

# Map electrodes to closest surface points
tree_h = cKDTree(fs_coords)
_, fs_indices = tree_h.query(human_coords)

# Create vertex color array
vertex_colors_human = np.zeros(len(fs_coords))
vertex_colors_human[fs_indices] = activity

# Visualize
plotter = pv.Plotter()
mesh_h = pv.PolyData(fs_coords, fs_faces)
mesh_h["Activity"] = vertex_colors_human
plotter.add_mesh(mesh_h, scalars="Activity", cmap="coolwarm", show_edges=False)
plotter.add_title("Human Analogue of Macaque ECoG")
plotter.show()
